In [14]:
import sys 
import os
import numpy as np
import itertools
from dataclasses import dataclass, replace, field
from matplotlib import pyplot as plt
from typing import Optional, Mapping, Dict, Tuple, TypeVar, Iterable, Sequence, Callable, Iterator, List
from scipy.stats import norm




from rl.function_approx import Tabular
from rl.chapter3.simple_inventory_mdp_cap import InventoryState, SimpleInventoryMDPCap
from rl.markov_decision_process import TransitionStep
from rl.approximate_dynamic_programming import ValueFunctionApprox
from rl.distribution import Choose
from rl.iterate import last
from rl.markov_process import MarkovProcess, NonTerminal, State
from rl.distribution import Gamma, Distribution, Constant
from rl.dynamic_programming import policy_iteration_result
from rl.policy import FiniteDeterministicPolicy
import rl.td
import rl.monte_carlo
import rl.td_lambda
import rl.iterate as iterate
from itertools import islice
from collections import defaultdict
from pprint import pprint







user_capacity = 2
user_poisson_lambda = 1.0
user_holding_cost = 1.0
user_stockout_cost = 10.0
user_gamma = 0.9

si_mdp = SimpleInventoryMDPCap(capacity = user_capacity,
                               poisson_lambda = user_poisson_lambda,
                               holding_cost = user_holding_cost,
                               stockout_cost = user_stockout_cost
                              )

opt_vf_vi, opt_policy_pi = policy_iteration_result(
    si_mdp, gamma = user_gamma)

print(si_mdp)

From State InventoryState(on_hand=0, on_order=0):
  With Action 0:
    To [State InventoryState(on_hand=0, on_order=0) and Reward -10.000] with Probability 1.000
  With Action 1:
    To [State InventoryState(on_hand=0, on_order=1) and Reward -10.000] with Probability 1.000
  With Action 2:
    To [State InventoryState(on_hand=0, on_order=2) and Reward -10.000] with Probability 1.000
From State InventoryState(on_hand=0, on_order=1):
  With Action 0:
    To [State InventoryState(on_hand=1, on_order=0) and Reward -0.000] with Probability 0.368
    To [State InventoryState(on_hand=0, on_order=0) and Reward -3.679] with Probability 0.632
  With Action 1:
    To [State InventoryState(on_hand=1, on_order=1) and Reward -0.000] with Probability 0.368
    To [State InventoryState(on_hand=0, on_order=1) and Reward -3.679] with Probability 0.632
From State InventoryState(on_hand=0, on_order=2):
  With Action 0:
    To [State InventoryState(on_hand=2, on_order=0) and Reward -0.000] with Probability

Below is our known optimal policy via DP

In [20]:
pprint(opt_vf_vi)
print(opt_policy_pi)

{NonTerminal(state=InventoryState(on_hand=1, on_order=1)): -28.991900091403522,
 NonTerminal(state=InventoryState(on_hand=0, on_order=1)): -27.660960231637496,
 NonTerminal(state=InventoryState(on_hand=1, on_order=0)): -28.660960231637496,
 NonTerminal(state=InventoryState(on_hand=0, on_order=2)): -27.991900091403522,
 NonTerminal(state=InventoryState(on_hand=2, on_order=0)): -29.991900091403522,
 NonTerminal(state=InventoryState(on_hand=0, on_order=0)): -34.89485578163003}
For State InventoryState(on_hand=0, on_order=0): Do Action 1
For State InventoryState(on_hand=0, on_order=1): Do Action 1
For State InventoryState(on_hand=0, on_order=2): Do Action 0
For State InventoryState(on_hand=1, on_order=0): Do Action 1
For State InventoryState(on_hand=1, on_order=1): Do Action 0
For State InventoryState(on_hand=2, on_order=0): Do Action 0



# Now lets use Tabular MC Control to try and find the above


In [15]:
def get_vf_and_policy_from_qvf(mdp, qvf):
    opt_vf = {s: max(qvf[(s,a)] for a in mdp.actions(s))
             for s in mdp.non_terminal_states
             }
    pol = {}
    for s in mdp.non_terminal_states:
        q_vals = {a: qvf[(s,a)] for a in mdp.actions(s)}
        pol[s.state] = max(q_vals, key = q_vals.get)
        
    opt_policy = FiniteDeterministicPolicy(pol)
    return opt_vf, opt_policy


episode_length_tolerance = 1e-5
eps_as_func_of_episodes = lambda k : k ** -1

initial_qvf_dict = {(s,a): 0. for s in si_mdp.non_terminal_states for a in si_mdp.actions(s)}
qvfs = rl.monte_carlo.glie_mc_tabular_control(si_mdp, Choose(si_mdp.non_terminal_states),
                               initial_qvf_dict, user_gamma,
                               eps_as_func_of_episodes, 
                               episode_length_tolerance)


num_episodes = 10000
final_qvf = iterate.last(itertools.islice(qvfs, num_episodes))
opt_vf, opt_policy = get_vf_and_policy_from_qvf(si_mdp, final_qvf)
pprint(opt_vf)
print(opt_policy)

{NonTerminal(state=InventoryState(on_hand=0, on_order=0)): -35.503215118052246,
 NonTerminal(state=InventoryState(on_hand=2, on_order=0)): -30.336675438964534,
 NonTerminal(state=InventoryState(on_hand=1, on_order=1)): -29.35315787037659,
 NonTerminal(state=InventoryState(on_hand=1, on_order=0)): -28.939052659730574,
 NonTerminal(state=InventoryState(on_hand=0, on_order=1)): -27.943732795253123,
 NonTerminal(state=InventoryState(on_hand=0, on_order=2)): -28.335770099725305}
For State InventoryState(on_hand=0, on_order=0): Do Action 2
For State InventoryState(on_hand=0, on_order=1): Do Action 1
For State InventoryState(on_hand=0, on_order=2): Do Action 0
For State InventoryState(on_hand=1, on_order=0): Do Action 1
For State InventoryState(on_hand=1, on_order=1): Do Action 0
For State InventoryState(on_hand=2, on_order=0): Do Action 0



It looks like our answers are reasonable when we compare them to the DP solution above, with each value function within $2\%$. However, it seems that the optimal action differs for one state, the (0,0) state. The fact that the optimal value functions don't differ that much seem to tell me that the optimal policy choice of Action 1 over Action 2 leads to very negligible differences in the reward structure, and since Tabular MC Control doesn't have the best convergence properties, that we've reached a "good enough" optimal policy, given the facts. 

# And now Q-Learning

In [24]:
max_episode_length = 100
epsilon = 0.2
qvfs = rl.td.tabular_q_learning(si_mdp, 
                                policy_from_q= lambda f, m : rl.monte_carlo.epsilon_greedy_policy_tab(
                                    q = f,
                                    mdp = si_mdp,
                                    epsilon = epsilon),
                                states = Choose(si_mdp.non_terminal_states),
                                approx_0 = initial_qvf_dict,
                                gamma = user_gamma,
                                max_episode_length = max_episode_length
                               )
final_qvf_learn = iterate.last(itertools.islice(qvfs, num_episodes))
opt_vf, opt_policy = get_vf_and_policy_from_qvf(si_mdp,final_qvf_learn)


pprint(opt_vf)
print(opt_policy)

{NonTerminal(state=InventoryState(on_hand=0, on_order=0)): -34.57887962360952,
 NonTerminal(state=InventoryState(on_hand=2, on_order=0)): -30.16224382407824,
 NonTerminal(state=InventoryState(on_hand=1, on_order=1)): -28.693013053803757,
 NonTerminal(state=InventoryState(on_hand=1, on_order=0)): -28.35708328317616,
 NonTerminal(state=InventoryState(on_hand=0, on_order=1)): -27.329398206917812,
 NonTerminal(state=InventoryState(on_hand=0, on_order=2)): -27.579792381783722}
For State InventoryState(on_hand=0, on_order=0): Do Action 1
For State InventoryState(on_hand=0, on_order=1): Do Action 1
For State InventoryState(on_hand=0, on_order=2): Do Action 0
For State InventoryState(on_hand=1, on_order=0): Do Action 1
For State InventoryState(on_hand=1, on_order=1): Do Action 0
For State InventoryState(on_hand=2, on_order=0): Do Action 0



Now we'll test these functions on the OVF/OP obtained by ADP on AssetAllocDiscrete

In [39]:
from rl.chapter7.asset_alloc_discrete import AssetAllocDiscrete
from rl.distribution import Gaussian
from rl.function_approx import DNNSpec, DNNApprox

steps = 4
mu = 0.13
sig = 0.2
r = 0.07
a = 1.0
init_wealth = 1.0
init_wealth_stdev = 0.1

excess = mu - r
var = sig**2
base_alloc = excess / (a*var)


risky_ret = [Gaussian(μ = mu, σ = sig) for _ in range(steps)]
riskless_ret = [r for _ in range(steps)]
utility_function = lambda x: -np.exp(-a*x) / a
alloc_choices = np.linspace(2/3 * base_alloc, 4/3 * base_alloc, 11)

feature_funcs = [
    lambda _: 1,
    lambda w_x: w_x[0],
    lambda w_x: w_x[1],
    lambda w_x: w_x[1] * w_x[1]
]


dnn = DNNSpec(
    neurons = [],
    bias = False,
    hidden_activation = lambda x:x,
    hidden_activation_deriv = lambda y: np.ones_like(y),
    output_activation = lambda x: -np.sign(a) * np.exp(-x),
    output_activation_deriv = lambda y: -y
)

init_wealth_distr = Gaussian(μ = init_wealth, σ = init_wealth_stdev)
aad = AssetAllocDiscrete(
    risky_return_distributions = risky_ret,
    riskless_returns=riskless_ret,
    utility_func=utility_function,
    risky_alloc_choices=alloc_choices,
    feature_functions=feature_funcs,
    dnn_spec=dnn,
    initial_wealth_distribution=init_wealth_distr
)


In [ ]:
episode_length_tolerance = 1e-5
eps_as_func_of_episodes = lambda k : k ** -1

initial_qvf_dict = 



aad_mdp = aad.get_mdp(0)


aad_qvfs = rl.monte_carlo.glie_mc_tabular_control(aad_mdp, Choose(aad_mdp.non_terminal_states),
                               initial_qvf, user_gamma,
                               eps_as_func_of_episodes, 
                               episode_length_tolerance)


num_episodes = 10000
aad_final_qvf = iterate.last(itertools.islice(qvfs, num_episodes))
aad_opt_vf, aad_opt_policy = get_vf_and_policy_from_qvf(aad_mdp, aad_final_qvf)
pprint(aad_opt_vf)
print(aad_opt_policy)


